In [2]:
import os
from nltk.tag import StanfordNERTagger as NERTagger
from nltk.tag import StanfordPOSTagger as POSTagger
ner_jar = 'StanfordNLP/jars/stanford-ner.jar'
ner_model = 'StanfordNLP/models/english.all.3class.distsim.crf.ser.gz'
pos_jar = 'StanfordNLP/jars/stanford-postagger.jar'
pos_model = 'StanfordNLP/models/english-bidirectional-distsim.tagger'
ner_tagger = NERTagger(ner_model, ner_jar)
pos_tagger = POSTagger(pos_model, pos_jar)

In [6]:
import pickle
dic_freq = []
files = os.listdir('pickles/apple_inc/')
for f in files:
    with open('pickles/apple_inc/' + f, 'rb') as f:
        dic_freq.append(pickle.load(f))
word_list = []
for dic in dic_freq:
    word_list.append([i[0] for i in dic])
for wl in word_list:
    print(wl[:20])
pos_tagger.tag(word_list[0][:20])

['the', 'and', 'Company', 'for', 'its', 'The Company', 'with', 'that', 'are', 'sales', 'net', 'September', 'products', 'not', 'other', 'from', 'financial', 'may', 'these', 'tax']
['the', 'and', 'Company', 'for', 'any', 'that', 'with', 'are', 'its', 'may', 'other', 'The Company', 'such', 'not', 'under', 'shall', 'Plan', 'which', 'sales', 'net']
['the', 'and', 'Company', 'for', 'are', 'its', 'with', 'that', 'The Company', 'sales', 'net', 'September', 'products', 'financial', 'from', 'other', 'may', 'not', 'which', 'billion']
['the', 'and', 'Company', 'for', 'that', 'its', 'with', 'are', 'The Company', 'any', 'other', 'sales', 'may', 'net', 'September', 'not', 'such', 'financial', 'from', 'products']
['the', 'and', 'Company', 'for', 'with', 'that', 'its', 'are', 'The Company', 'sales', 'other', 'any', 'net', 'not', 'September', 'may', 'from', 'such', 'products', 'which']


[('the', 'DT'),
 ('and', 'CC'),
 ('Company', 'NNP'),
 ('for', 'IN'),
 ('its', 'PRP$'),
 ('The', 'NNP'),
 ('Company', 'NNP'),
 ('with', 'IN'),
 ('that', 'DT'),
 ('are', 'VBP'),
 ('sales', 'NNS'),
 ('net', 'JJ'),
 ('September', 'NNP'),
 ('products', 'NNS'),
 ('not', 'RB'),
 ('other', 'JJ'),
 ('from', 'IN'),
 ('financial', 'JJ'),
 ('may', 'MD'),
 ('these', 'DT'),
 ('tax', 'NN')]

In [15]:
org_list = []
org_set = set()
loc_list = []
loc_set = set()
per_list = []
per_set = set()
spe_list = []
spe_set = []
for wl in word_list:
    wl_compound = list(filter(lambda word: ' ' in word, wl))
    spe_list.append(wl_compound)
    spe_set += wl_compound
    #wl = list(filter(lambda word: ' ' not in word, wl))
    dic_ner = dict(ner_tagger.tag(wl))
    org_list.append([])
    loc_list.append([])
    per_list.append([])
    for word, ner in dic_ner.items():
        if len(word) < 3:
            continue
        if ner == 'ORGANIZATION':
            org_list[-1].append(word)
            org_set.add(word)
        if ner == 'LOCATION':
            loc_list[-1].append(word)
            loc_set.add(word)
        if ner == 'PERSON':
            per_list[-1].append(word)
            per_set.add(word)
spe_set = set(spe_set)
print(list(org_set)[:20])
print(list(loc_set)[:20])
print(list(per_set)[:20])
print(list(spe_set)[:20])

['March', 'Treasury', 'Our', 'Inventory', 'GORE', 'Force', 'Mellon', 'Organizations', 'Non-', 'Intellectual', 'Monte', 'Released', 'Comments', 'Those', 'Street', 'Investments', 'Seasonality', 'Including', 'Xcode', 'Contents']
['Europe', 'Silicon', 'Middle', 'Taiwan', 'Overall', 'Retina', 'China', 'East', 'Santa', 'Hong', 'While', 'Washington', 'Kong', 'America', 'Japan', 'County', 'Asia-Pacific', 'Clara', 'Valley', 'Ireland']
['Angela', 'DREXLER', 'JUNG', 'Susan', 'Andrea', 'ANDREA', 'LEVINSON', 'Siri', 'William', 'Jung', 'Law', 'ROBERT', 'Riccio', 'Robert', 'SUSAN', 'Sewell', 'Jeffrey', 'Daniel', 'Chris', 'Cook']
['Hedge Instruments Fair Value Derivative', 'Contents Information', 'Supplementary Data”', 'Counterparts This Award Agreement', 'Related Persons”', 'Name Title Date', 'Equity Securities The Company', 'Other Intangible Assets The Company', 'Open Market Taxes Related', 'Quattro Wireless Inc', 'Total Earnings', 'Fiscal Period The Company', 'Other Contingencies As', 'Mac Develope

In [16]:
adj_list = []
adj_set = set()
for wl in word_list:
    dic_pos = dict(pos_tagger.tag(wl[:200]))
    adj_list.append([])
    for word, pos in dic_pos.items():
        if len(word) < 3:
            continue
        if pos == 'JJ':
            adj_list[-1].append(word)
            adj_set.add(word)
print(list(adj_set)[:20])

['new', 'deferred', 'various', 'material', 'otherwise', 'other', 'total', 'available', 'pursuant', 'future', 'certain', 'Other', 'registrant', 'issued', 'effective', 'due', 'gross', 'outstanding', 'Registrant', 'included']


In [17]:
import numpy as np
import pandas as pd
org_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in org_set] for dic in dic_freq])
org_matrix = pd.DataFrame(org_matrix, columns = org_set)
org_matrix

,March,Treasury,Our,Inventory,GORE,Force,Mellon,Organizations,Non-,Intellectual,...,California,Related,Restricted,Document,Owners,Flickr,Component,Rest,Touch,May
0,6,0,0,0,0,0,1,1,0,0,...,16,1,1,6,0,1,1,9,0,9
1,9,0,0,0,1,0,1,0,0,0,...,20,3,6,6,1,0,0,8,1,12
2,6,0,0,0,0,0,1,1,1,0,...,12,2,0,6,0,0,0,15,1,8
3,7,0,0,1,1,0,1,0,0,0,...,19,3,2,6,0,0,0,11,0,9
4,4,0,0,0,0,0,1,0,0,0,...,25,3,0,6,0,1,0,16,0,7


In [111]:
per_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in per_set] for dic in dic_freq])
per_matrix = pd.DataFrame(per_matrix, columns = per_set)
per_matrix

,WILLIAM,Susan,Chris,JUNG,Ahrendts,Philip,Drexler,Andrea,William,DREXLER,...,Millard,ARTHUR,Arthur,WAGNER,Maestri,Campbell,Robert,CHRIS,Angela,Jeffrey
0,1,0,0,0,0,1,0,1,2,0,...,1,1,1,0,1,0,1,0,0,1
1,0,1,1,1,0,0,0,1,0,0,...,0,1,1,0,0,0,1,1,0,0
2,0,1,1,0,0,1,0,1,0,0,...,0,1,1,0,0,0,1,1,0,0
3,0,1,1,1,0,1,0,1,0,0,...,0,1,1,0,0,0,1,1,0,0
4,0,1,0,0,0,1,0,1,0,0,...,1,1,1,0,0,0,1,0,0,1


In [112]:
loc_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in loc_set] for dic in dic_freq])
loc_matrix = pd.DataFrame(loc_matrix, columns = loc_set)
loc_matrix

,While,East,Ireland,Santa,Europe,Retina,Taiwan,Japan,County,Washington,Valley,Hong,America,Asia-Pacific,Middle,China,Overall,Silicon,Clara,Kong
0,1,0,10,0,21,6,4,9,0,2,0,5,4,2,0,10,1,0,0,5
1,0,0,23,2,11,1,4,6,2,2,0,0,3,0,0,10,0,0,2,0
2,1,0,20,0,16,4,3,12,0,2,0,0,4,0,0,9,0,0,0,0
3,0,0,20,2,14,3,3,7,2,2,0,4,3,0,0,8,0,0,2,4
4,2,0,13,2,19,7,3,15,2,2,0,1,4,0,0,7,0,0,2,1


In [113]:
spe_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in spe_set] for dic in dic_freq])
spe_matrix = pd.DataFrame(spe_matrix, columns = spe_set)
spe_matrix

,Photo Library Family Sharing Find My,Financial Disclosure None Item,Store AppleCare,Exchange Act Rules,Numbers® The Company,Retail: Net,Apple Inc CONSOLIDATED STATEMENTS OF CASH FLOWS In,U S During,Identifying Performance Obligations,Non-Designated Derivatives Derivatives,...,Expense The,AS ADOPTED PURSUANT TO SECTION,European Union,Share-based Awards,Andrea Jung Director October,Performance Goals,Benefit Plans Stock Plans,Increase/ Decrease,U S Internal Revenue Code,Middle East
0,0,1,1,4,0,1,0,1,0,0,...,3,1,0,0,1,0,1,0,0,3
1,1,1,0,4,1,0,1,0,1,1,...,0,1,0,2,0,12,0,1,1,3
2,1,1,0,4,0,0,0,0,0,1,...,0,1,1,0,1,0,0,0,0,3
3,1,1,0,4,1,0,0,0,1,1,...,0,1,0,0,0,0,0,1,0,3
4,0,1,4,4,0,1,0,1,0,0,...,3,1,1,0,1,0,1,0,0,3


In [18]:
adj_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in adj_set] for dic in dic_freq])
adj_df = pd.DataFrame(adj_matrix, columns = adj_set)
adj_df

,new,deferred,various,material,otherwise,other,total,available,pursuant,future,...,derivative,related,marketable,financial,used,retail,internal,digital,significant,foreign
0,67,35,32,43,11,179,50,45,14,69,...,42,108,38,168,35,41,44,39,69,85
1,73,42,25,50,61,294,57,58,86,79,...,47,118,33,187,50,31,48,36,54,121
2,68,33,26,46,14,161,48,45,17,75,...,45,97,34,175,38,40,46,40,55,102
3,63,35,22,47,42,221,53,41,54,75,...,48,111,34,187,45,32,47,40,58,111
4,73,34,32,44,38,246,66,45,56,73,...,48,129,36,171,43,36,44,39,72,89


In [23]:
adj_comatrix = np.matmul(np.transpose(adj_matrix), adj_matrix)
adj_comatrix = pd.DataFrame(adj_comatrix, columns = adj_set, index = adj_set)
adj_comatrix

,new,deferred,various,material,otherwise,other,total,available,pursuant,future,...,derivative,related,marketable,financial,used,retail,internal,digital,significant,foreign
new,23740,12342,9459,15832,11562,76284,18932,16177,15862,25544,...,15833,38856,12037,61071,14553,12374,15753,13328,21215,34954
deferred,12342,6459,4886,8264,6171,40025,9827,8461,8457,13315,...,8241,20208,6252,31868,7616,6401,8215,6923,10976,18334
various,9459,4886,3833,6264,4381,29998,7551,6402,6020,10119,...,6281,15498,4825,24187,5724,4983,6246,5316,8568,13687
material,15832,8264,6264,10610,7813,51014,12603,10812,10686,17104,...,10594,25899,8030,40937,9760,8241,10553,8913,14091,23530
otherwise,11562,6171,4381,7813,7246,40787,9433,8095,10034,12552,...,7799,19308,5703,30057,7491,5614,7702,6347,9995,17788
other,76284,40025,29998,51014,40787,253755,61385,52483,56237,82185,...,50997,125906,38348,196618,47606,38821,50605,42439,67612,113636
total,18932,9827,7551,12603,9433,61385,15218,12859,12976,20346,...,12651,31179,9591,48656,11647,9809,12539,10616,16994,27800
available,16177,8461,6402,10812,8095,52483,12859,11120,11117,17422,...,10769,26425,8168,41643,9965,8375,10741,9038,14330,23989
pursuant,15862,8457,6020,10686,10034,56237,12976,11117,13933,17173,...,10675,26527,7800,41083,10274,7664,10528,8666,13709,24308
future,25544,13315,10119,17104,12552,82185,20346,17422,17173,27581,...,17090,41791,12957,65998,15729,13306,17015,14382,22758,37896
